In this notebook, the steps to create an integrated walk and transit network are shown. 

Note: this notebook should be run in GOOGLE COLAB, as Visual Studio has shown problems when creating the integrated network.

In [ ]:
!pip install urbanaccess

In [ ]:
import numpy as np
import os
import glob
import statistics
from pathlib import Path
import re
import requests
from io import StringIO
import random
import pdb
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import time
import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network
#import pandas as pd
from datetime import datetime
import pandana as pdn

In [ ]:
feeds.add_feed(add_dict={'dbus': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/dbus/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_areizaga': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_areizaga/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_arrasate': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_arrasate/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_auif_urb': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_auif_urb/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_eibar': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_eibar/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_ekialdebus': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_ekialdebus/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_euskotren_bus': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_euskotren_bus/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_garayar': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_garayar/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_gipuzkoana': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_gipuzkoana/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_goierrialdea': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_goierrialdea/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_hernani': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_hernani/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_lasarte': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_lasarte/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_oiartzun': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_oiartzun/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_pesa': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_pesa/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_renteria': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_renteria/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_tbh': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_tbh/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_tolosaldea': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_tolosaldea/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_tsst': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_tsst/google_transit.zip'})
feeds.add_feed(add_dict={'lurraldebus_zarautz': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/ATTG/lurraldebus_zarautz/google_transit.zip'})
feeds.add_feed(add_dict={'Euskotren': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/Euskotren/google_transit.zip'})
feeds.add_feed(add_dict={'Renfe': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/Renfe/google_transit.zip'})
feeds.add_feed(add_dict={'Renfe_cercanias': 'https://www.geo.euskadi.eus/cartografia/DatosDescarga/Transporte/Moveuskadi/Renfe_Cercanias/google_transit.zip'})

#feeds.add_feed(add_dict={'SCP_routes': '.\data\gtfsfeed_zips\routes_EZ_companies.zip'})
# I download all these feeds, 70sg
gtfsfeeds.download()

In [ ]:
# Load GTFS data into an UrbanAcess transit data object
validation = True
verbose = True
# bbox for Gipuzkoa
Lat_min = 42.904155
Long_min = -2.621987
Lat_max = 43.403070
Long_max = -1.740334
bbox = (Long_min,Lat_min,Long_max,Lat_max,)
remove_stops_outsidebbox = True
append_definitions = True

#filen = 'transit_ped_net.h5'
gtfsfeed_path = './data/gtfsfeed_text'
networks_path = './data/input_data_MCM/networks/'
transit_path = './data/input_data_MCM/transit_together_24h/'

# Definir los valores dados
timeranges = [
    ['00:00:00', '01:00:00'], ['01:00:00', '02:00:00'], ['02:00:00', '03:00:00'], ['03:00:00', '04:00:00'],
    ['04:00:00', '05:00:00'], ['05:00:00', '06:00:00'], ['06:00:00', '07:00:00'], ['07:00:00', '08:00:00'],
    ['08:00:00', '09:00:00'], ['09:00:00', '10:00:00'], ['10:00:00', '11:00:00'], ['11:00:00', '12:00:00'],
    ['12:00:00', '13:00:00'], ['13:00:00', '14:00:00'], ['14:00:00', '15:00:00'], ['15:00:00', '16:00:00'],
    ['16:00:00', '17:00:00'], ['17:00:00', '18:00:00'], ['18:00:00', '19:00:00'], ['19:00:00', '20:00:00'],
    ['20:00:00', '21:00:00'], ['21:00:00', '22:00:00'], ['22:00:00', '23:00:00'], ['23:00:00', '24:00:00']
]
nombres_archivos = [
    'transit_0001.h5', 'transit_0102.h5', 'transit_0203.h5', 'transit_0304.h5',
    'transit_0405.h5', 'transit_0506.h5', 'transit_0607.h5', 'transit_0708.h5',
    'transit_0809.h5', 'transit_0910.h5', 'transit_1011.h5', 'transit_1112.h5',
    'transit_1213.h5', 'transit_1314.h5', 'transit_1415.h5', 'transit_1516.h5',
    'transit_1617.h5', 'transit_1718.h5', 'transit_1819.h5', 'transit_1920.h5',
    'transit_2021.h5', 'transit_2122.h5', 'transit_2223.h5', 'transit_2324.h5'
]

# The 08:00:00 to 20:00:00 time range is selected as it is representative of the day
timeranges=[['08:00:00', '20:00:00']] # elegimos los timeranges que esten en nuestra franja
nombres_archivos=['transit_0820.h5'] # elegimos los nombres que correspondan a la franja
print(timeranges)
print(nombres_archivos)

In [ ]:
!pip install pandas==1.5.3

In [ ]:
for timerange, nombre_archivo in zip(timeranges, nombres_archivos):

        filen = f'{nombre_archivo}'
        # Create transit network
        urbanaccess_net = ua.network.ua_network

        # Create new transit network from new feeds ##############################################
        loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(gtfsfeed_path=gtfsfeed_path,
                                                validation=validation,
                                                verbose=verbose,
                                                bbox=bbox,
                                                remove_stops_outsidebbox=remove_stops_outsidebbox,
                                                append_definitions=append_definitions)

        print("Type of loaded_feeds:", type(loaded_feeds))
        print("Attributes of loaded_feeds:", dir(loaded_feeds))

        # Create transit network
        ua.gtfs.network.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                        day='monday',
                                        timerange=timerange,
                                        calendar_dates_lookup=None)
        #############################################################################################

        if (os.path.isfile(networks_path + 'pedestrian_net.h5')):

            print('\nLoading saved networks...')
            ped_net = pdn.Network.from_hdf5(networks_path + 'pedestrian_net.h5')
            print('done!\n')

            # The following sh.. is really important!!! Pandana messes up data types (floats instead of int64!) ####################
            ped_net.nodes_df['id'] = ped_net.nodes_df.index
            ##########################################################################################################################

            # create pedestrian network from saved file ######################################
            ua.osm.network.create_osm_net(osm_edges=ped_net.edges_df, osm_nodes=ped_net.nodes_df, travel_speed_mph=3)
            print('done!\n')

        else:
            nodes, edges = ua.osm.load.ua_network_from_bbox(bbox=bbox, remove_lcn=True)

            # Create pedestrian network from scratch #########################################
            ua.osm.network.create_osm_net(osm_edges=edges, osm_nodes=nodes, travel_speed_mph=3)

            ped_net_pdn = pdn.network.Network(
                                    nodes["x"],
                                    nodes["y"],
                                    edges["from"],
                                    edges["to"],
                                    edges[["weight","distance"]])

            ped_net_pdn.save_hdf5(networks_path + 'pedestrian_net.h5')

        print('\nIntegrating all networks...')
        # Integrate pedestrian and tansit network
        ua.network.integrate_network(urbanaccess_network=urbanaccess_net,  # do we need this?
                                    headways=False)

        # Add average headways to network travel time
        ua.gtfs.headways.headways(gtfsfeeds_df=loaded_feeds,
                            headway_timerange=timerange)
        loaded_feeds.headways

        ua.network.integrate_network(urbanaccess_network=urbanaccess_net,
                                headways=True,
                                urbanaccess_gtfsfeeds_df=loaded_feeds,
                                headway_statistic='mean')
        print('successfully integrated existing and newly created networks!')

        #print(urbanaccess_net.net_edges["from_int"].head())
        #urbanaccess_net.net_nodes.set_index('ID', inplace= True)
        transit_ped_net_pdn = pdn.network.Network(
                                urbanaccess_net.net_nodes["x"],
                                urbanaccess_net.net_nodes["y"],
                                urbanaccess_net.net_edges["from_int"],
                                urbanaccess_net.net_edges["to_int"],
                                urbanaccess_net.net_edges[["weight"]])
                                #urbanaccess_net.net_edges[["weight","distance"]])

        transit_ped_net_pdn.save_hdf5(transit_path + filen)